In [1]:
# dependencies
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
%matplotlib inline
# import dependencies
from getpass import getpass
import sqlalchemy
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, text
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf

In [2]:
# SQL Alchemy connection from RDS
secret = getpass('Enter the secret value: ')

args ={
    'host':"ogdataset.c11hekhsylui.us-west-1.rds.amazonaws.com",
    'port':'5432',
    'database':"og_dataset",
    'user':"attritionProject",
    'password':secret
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**args))
connection = engine.connect()

Enter the secret value: ········


/Users/stephen/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# read in table from database
attrition_df = pd.read_sql('SELECT * FROM new_encoded_data', connection)
attrition_df

,Age,Attrition,Business Travel,Department,Distance from Home,Education Level,Education Field,Environment Satisfaction,Gender,Job Involvement,...,Performance Rating,Relationship Satisfaction,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,1,2,2,1,2,1,2,0,3,...,3,1,0,8,0,1,6,4,0,5
1,49,0,1,1,8,1,1,3,1,2,...,4,4,1,10,3,3,10,7,1,7
2,37,1,2,1,2,2,4,4,1,2,...,3,2,0,7,3,3,0,0,0,0
3,33,0,1,1,3,4,1,4,0,3,...,3,3,0,8,3,3,8,7,3,0
4,27,0,2,1,2,1,3,1,1,3,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,1,1,23,2,3,3,1,4,...,3,3,1,17,3,3,5,2,0,3
1466,39,0,2,1,6,1,3,4,1,2,...,3,1,1,9,5,3,7,7,1,7
1467,27,0,2,1,4,3,1,2,1,4,...,4,2,1,6,0,3,6,2,0,3
1468,49,0,1,2,2,3,3,4,1,2,...,3,4,0,17,3,2,9,6,0,8


In [4]:
# check version number
import imblearn
print(imblearn.__version__)

0.7.0


In [5]:
# Split the preprocessed data into a training and testing dataset
# Split our preprocessed data into our features and target arrays
X = attrition_df.drop("Attrition",1)
y = attrition_df["Attrition"]
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, test_size= 0.20, train_size= 0.80, stratify =y)

/Users/stephen/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler# define oversampling strategy
# define oversampling strategy
#oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
#X_over, y_over = oversample.fit_resample(X_train, y_train)
# summarize class distribution
#print(Counter(y_over))

In [7]:
# Create a StandardScaler instances
#scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_over)

# Scale the data
#X_train_scaled = X_scaler.transform(X_over)
#X_test_scaled = X_scaler.transform(X_test)

In [8]:
attrition_df.shape

(1470, 28)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [10]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#thrid hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='sigmoid'))
    
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=500)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/500


2022-06-04 17:29:31.126883: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


35/35 [==============================] - 0s 1ms/step - loss: 0.4757 - accuracy: 0.8294
Epoch 2/500
35/35 [==============================] - 0s 954us/step - loss: 0.4286 - accuracy: 0.8294
Epoch 3/500
35/35 [==============================] - 0s 961us/step - loss: 0.4041 - accuracy: 0.8294
Epoch 4/500
35/35 [==============================] - 0s 959us/step - loss: 0.3759 - accuracy: 0.8367
Epoch 5/500
35/35 [==============================] - 0s 959us/step - loss: 0.3512 - accuracy: 0.8575
Epoch 6/500
35/35 [==============================] - 0s 990us/step - loss: 0.3333 - accuracy: 0.8675
Epoch 7/500
35/35 [==============================] - 0s 945us/step - loss: 0.3175 - accuracy: 0.8748
Epoch 8/500
35/35 [==============================] - 0s 945us/step - loss: 0.3083 - accuracy: 0.8811
Epoch 9/500
35/35 [==============================] - 0s 959us/step - loss: 0.2983 - accuracy: 0.8848
Epoch 10/500
35/35 [==============================] - 0s 957us/step - loss: 0.2887 - accuracy: 0.8911
Epo

35/35 [==============================] - 0s 884us/step - loss: 0.0061 - accuracy: 1.0000
Epoch 82/500
35/35 [==============================] - 0s 900us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 83/500
35/35 [==============================] - 0s 876us/step - loss: 0.0053 - accuracy: 1.0000
Epoch 84/500
35/35 [==============================] - 0s 887us/step - loss: 0.0051 - accuracy: 1.0000
Epoch 85/500
35/35 [==============================] - 0s 903us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 86/500
35/35 [==============================] - 0s 885us/step - loss: 0.0045 - accuracy: 1.0000
Epoch 87/500
35/35 [==============================] - 0s 878us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 88/500
35/35 [==============================] - 0s 892us/step - loss: 0.0041 - accuracy: 1.0000
Epoch 89/500
35/35 [==============================] - 0s 878us/step - loss: 0.0039 - accuracy: 1.0000
Epoch 90/500
35/35 [==============================] - 0s 886us/step - loss: 0.0037 - accuracy: 

35/35 [==============================] - 0s 965us/step - loss: 4.5395e-04 - accuracy: 1.0000
Epoch 160/500
35/35 [==============================] - 0s 969us/step - loss: 4.4378e-04 - accuracy: 1.0000
Epoch 161/500
35/35 [==============================] - 0s 887us/step - loss: 4.3285e-04 - accuracy: 1.0000
Epoch 162/500
35/35 [==============================] - 0s 878us/step - loss: 4.2682e-04 - accuracy: 1.0000
Epoch 163/500
35/35 [==============================] - 0s 917us/step - loss: 4.1248e-04 - accuracy: 1.0000
Epoch 164/500
35/35 [==============================] - 0s 914us/step - loss: 4.0600e-04 - accuracy: 1.0000
Epoch 165/500
35/35 [==============================] - 0s 929us/step - loss: 3.9640e-04 - accuracy: 1.0000
Epoch 166/500
35/35 [==============================] - 0s 923us/step - loss: 3.8641e-04 - accuracy: 1.0000
Epoch 167/500
35/35 [==============================] - 0s 983us/step - loss: 3.7814e-04 - accuracy: 1.0000
Epoch 168/500
35/35 [==============================

35/35 [==============================] - 0s 948us/step - loss: 9.4034e-05 - accuracy: 1.0000
Epoch 236/500
35/35 [==============================] - 0s 1ms/step - loss: 9.2068e-05 - accuracy: 1.0000
Epoch 237/500
35/35 [==============================] - 0s 974us/step - loss: 9.0726e-05 - accuracy: 1.0000
Epoch 238/500
35/35 [==============================] - 0s 960us/step - loss: 8.9065e-05 - accuracy: 1.0000
Epoch 239/500
35/35 [==============================] - 0s 919us/step - loss: 8.7211e-05 - accuracy: 1.0000
Epoch 240/500
35/35 [==============================] - 0s 941us/step - loss: 8.5611e-05 - accuracy: 1.0000
Epoch 241/500
35/35 [==============================] - 0s 938us/step - loss: 8.4026e-05 - accuracy: 1.0000
Epoch 242/500
35/35 [==============================] - 0s 938us/step - loss: 8.2296e-05 - accuracy: 1.0000
Epoch 243/500
35/35 [==============================] - 0s 910us/step - loss: 8.0981e-05 - accuracy: 1.0000
Epoch 244/500
35/35 [==============================] 

35/35 [==============================] - 0s 903us/step - loss: 2.3721e-05 - accuracy: 1.0000
Epoch 312/500
35/35 [==============================] - 0s 915us/step - loss: 2.3320e-05 - accuracy: 1.0000
Epoch 313/500
35/35 [==============================] - 0s 908us/step - loss: 2.2907e-05 - accuracy: 1.0000
Epoch 314/500
35/35 [==============================] - 0s 892us/step - loss: 2.2518e-05 - accuracy: 1.0000
Epoch 315/500
35/35 [==============================] - 0s 907us/step - loss: 2.2066e-05 - accuracy: 1.0000
Epoch 316/500
35/35 [==============================] - 0s 897us/step - loss: 2.1738e-05 - accuracy: 1.0000
Epoch 317/500
35/35 [==============================] - 0s 903us/step - loss: 2.1388e-05 - accuracy: 1.0000
Epoch 318/500
35/35 [==============================] - 0s 917us/step - loss: 2.0934e-05 - accuracy: 1.0000
Epoch 319/500
35/35 [==============================] - 0s 944us/step - loss: 2.0658e-05 - accuracy: 1.0000
Epoch 320/500
35/35 [==============================

35/35 [==============================] - 0s 941us/step - loss: 6.3344e-06 - accuracy: 1.0000
Epoch 388/500
35/35 [==============================] - 0s 913us/step - loss: 6.2333e-06 - accuracy: 1.0000
Epoch 389/500
35/35 [==============================] - 0s 889us/step - loss: 6.1238e-06 - accuracy: 1.0000
Epoch 390/500
35/35 [==============================] - 0s 894us/step - loss: 6.0201e-06 - accuracy: 1.0000
Epoch 391/500
35/35 [==============================] - 0s 898us/step - loss: 5.9130e-06 - accuracy: 1.0000
Epoch 392/500
35/35 [==============================] - 0s 896us/step - loss: 5.8158e-06 - accuracy: 1.0000
Epoch 393/500
35/35 [==============================] - 0s 904us/step - loss: 5.7133e-06 - accuracy: 1.0000
Epoch 394/500
35/35 [==============================] - 0s 903us/step - loss: 5.6217e-06 - accuracy: 1.0000
Epoch 395/500
35/35 [==============================] - 0s 903us/step - loss: 5.5179e-06 - accuracy: 1.0000
Epoch 396/500
35/35 [==============================

35/35 [==============================] - 0s 932us/step - loss: 1.7206e-06 - accuracy: 1.0000
Epoch 464/500
35/35 [==============================] - 0s 927us/step - loss: 1.6923e-06 - accuracy: 1.0000
Epoch 465/500
35/35 [==============================] - 0s 881us/step - loss: 1.6637e-06 - accuracy: 1.0000
Epoch 466/500
35/35 [==============================] - 0s 896us/step - loss: 1.6326e-06 - accuracy: 1.0000
Epoch 467/500
35/35 [==============================] - 0s 892us/step - loss: 1.6059e-06 - accuracy: 1.0000
Epoch 468/500
35/35 [==============================] - 0s 888us/step - loss: 1.5829e-06 - accuracy: 1.0000
Epoch 469/500
35/35 [==============================] - 0s 877us/step - loss: 1.5518e-06 - accuracy: 1.0000
Epoch 470/500
35/35 [==============================] - 0s 882us/step - loss: 1.5263e-06 - accuracy: 1.0000
Epoch 471/500
35/35 [==============================] - 0s 886us/step - loss: 1.5049e-06 - accuracy: 1.0000
Epoch 472/500
35/35 [==============================